# House of Requests

Use the `Deck of Cards` API to play a game of BlackJack with a partner. Parameterize the `Deck of Cards` API `request urls` in order to create the deck of cards, as well as draw cards for the game.

In [95]:
import requests

# Function to calculate points
def calculate_points(cards):
    points = 0
    aces = 0
    for card in cards:
        if card['value'] in ["KING", "QUEEN", "JACK"]:
            points += 10
        elif card['value'] == "ACE":
            points += 11
            aces += 1
        else:
            points += int(card['value'])
    # Adjust for Aces
    while points > 21 and aces:
        points -= 10
        aces -= 1
    return points

# Function to decide whether to draw another card
def decide_to_draw(points):
    if points < 17:
        return "draw"
    elif points > 21:
        return "lose"
    elif points == 21:
        return "win"
    else:
        return "hold"


## Prep for the Game

In [96]:
# Declare request url to create deck id
create_shuffle_url = "https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=6"

# Execute create deck url
response = requests.get(create_shuffle_url)

### Parse JSON and extract `deck_id`

In [97]:
create_shuffle_url = "https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1"
response = requests.get(create_shuffle_url).json()
deck_id = response['deck_id']
print(deck_id)


i9jacr1lxpdv


### Declare request urls to draw cards and shuffle deck

In [98]:
# Declare draw_cards_url and shuffle_deck_url
# Use string interpolation to incorporate the deck_id
shuffle_deck_url = f"https://deckofcardsapi.com/api/deck/{deck_id}/shuffle/"
draw_cards_url = f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=2"


## Player 1 Turn

### Draw two cards

In [99]:
# Draw two cards
player_1_cards = requests.get(draw_cards_url).json()['cards']


### Parse and extract `value` and `suit` elements from JSON output, for each card

In [100]:
# Select returned card's value and suit (i.e. 3 of clubs)


# Print player cards
print("Player's Initial Cards:", [(card['value'], card['suit']) for card in player_1_cards])


Player's Initial Cards: [('6', 'DIAMONDS'), ('4', 'SPADES')]


### Decide whether to draw another card or skip to next player turn

In [101]:
player_points = calculate_points(player_1_cards)
decision = decide_to_draw(player_points)
print("Player's Points:", player_points, "| Decision:", decision)

if decision == "draw":
    draw_one_more_card_url = f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=1"
    new_card = requests.get(draw_one_more_card_url).json()['cards'][0]
    player_1_cards.append(new_card)
    player_points = calculate_points(player_1_cards)
    print("Player drew:", (new_card['value'], new_card['suit']))
    print("Player's New Points:", player_points)


Player's Points: 10 | Decision: draw
Player drew: ('3', 'SPADES')
Player's New Points: 13


In [102]:
if player_points > 21:
    print("Player busts!")
elif player_points == 21:
    print("Player hits 21!")
else:
    print("Player holds at", player_points)

Player holds at 13


## Player 2 / Dealer Turn

In [103]:
# Draw 2 cards for the dealer
draw_cards_url = f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=2"
dealer_cards = requests.get(draw_cards_url).json()['cards']
print("Dealer's Initial Cards:", [(card['value'], card['suit']) for card in dealer_cards])


Dealer's Initial Cards: [('ACE', 'SPADES'), ('QUEEN', 'DIAMONDS')]


In [104]:
# Calculate dealer points and make decision based on player's outcome
dealer_points = calculate_points(dealer_cards)

# If player busts, dealer does not need to draw more cards
if player_points > 21:
    print("Player has busted. Dealer wins by default.")
else:
    print("Dealer's Points:", dealer_points)
    while dealer_points < 17:
        # Dealer draws if points are less than 17
        draw_one_more_card_url = f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=1"
        new_card = requests.get(draw_one_more_card_url).json()['cards'][0]
        dealer_cards.append(new_card)
        dealer_points = calculate_points(dealer_cards)
        print("Dealer drew:", (new_card['value'], new_card['suit']))
        print("Dealer's New Points:", dealer_points)


Dealer's Points: 21


In [105]:
# Determine the outcome of the game
if player_points > 21:
    print("Player busts! House wins!")
elif dealer_points > 21:
    print("Dealer busts! Player wins!")
elif dealer_points > player_points:
    print("House wins with", dealer_points, "points.")
elif dealer_points < player_points:
    print("Player wins with", player_points, "points.")
else:
    print("It's a tie! Both player and dealer have", dealer_points, "points.")


House wins with 21 points.
